In [1]:
# importing sys
import sys
# adding src to the system path
sys.path.insert(0, './src')

import os
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from DataSetsLoader import DataSetsLoader
from PointNetFactory import PointNetFactory

C:\Users\binwe\anaconda3\envs\tensorflow26_p39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Global variables
NUM_POINTS = 2048
NUM_CLASSES = 10
NUM_POINT_AXIS = 3
BATCH_SIZE = 32

# Load Model
point_net_factory = PointNetFactory()
model = point_net_factory.create_model(NUM_POINTS, NUM_POINT_AXIS, NUM_CLASSES)
model.load_weights("./model.h5")

In [4]:
# Load Datasets
datasets_loader = DataSetsLoader()
train_points, test_points, train_labels, test_labels, CLASS_MAP = datasets_loader.transform_to_tensorflow_dataset()

processing class: bathtub
processing class: bed
processing class: chair
processing class: night_stand
processing class: sofa
processing class: table
processing class: toilet


In [81]:
# Shuffle and Augment Datasets
def augment(points, label):
        # jitter points
        points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
        # shuffle points
        points = tf.random.shuffle(points)
        return points, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_points, test_labels))

train_dataset = train_dataset.shuffle(len(train_points)).map(augment).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

In [91]:
dataset = test_dataset
total_count = 0
correct_count = 0

for element in dataset.as_numpy_iterator():
    points = element[0]
    labels = element[1]
    pred = model.predict(points)
    pred = tf.math.argmax(pred, -1)
    for index, point in enumerate(points):
        pred = model.predict(points)
        pred = tf.math.argmax(pred, -1)
        total_count += 1
        if labels[index] == pred[index]:
            correct_count += 1
            
accuracy  = correct_count / total_count
print("Accuracy  is {}".format(accuracy))

Accuracy  is 0.8160377358490566
